In [4]:
import argparse
import base64
import json

import numpy as np
import socketio
import eventlet
import eventlet.wsgi
import time
from PIL import Image
from PIL import ImageOps
from flask import Flask, render_template
from io import BytesIO

from keras.models import model_from_json
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array

# Fix error with Keras and TensorFlow
import tensorflow as tf
tf.python.control_flow_ops = tf

sio = socketio.Server()
app = Flask(__name__)
model = None
prev_image_array = None

@sio.on('telemetry')
def telemetry(sid, data):
    # The current steering angle of the car
    steering_angle = data["steering_angle"]
    # The current throttle of the car
    throttle = data["throttle"]
    # The current speed of the car
    speed = data["speed"]
    # The current image from the center camera of the car
    imgString = data["image"]
    image = Image.open(BytesIO(base64.b64decode(imgString)))
    image_array = np.asarray(image)
    #transformed_image_array = cv2.resize((image_array[30:135, :, :]),(200,66))

    #add
    images = images[40:,:,:]
    images = cv2.resize(images,(220,66),interpolation=cv2.INTER_CUBIC)
    images =cv2.cvtColor(images,cv2.COLOR_BGR2YUV)
    images= images.astype('float32')

    #transformed_image_array = image_array[None, :, :, :]
    # This model currently assumes that the features of the model are just the images. Feel free to change this.
    steering_angle = float(model.predict(images, batch_size=1))
    print(steering_angle)
    # The driving model currently just outputs a constant throttle. Feel free to edit this.
    throttle = 0.5
    print(steering_angle, throttle)
    send_control(steering_angle, throttle)
    

@sio.on('connect')
def connect(sid, environ):
    print("connect ", sid)
    send_control(0, 0)


def send_control(steering_angle, throttle):
    sio.emit("steer", data={
    'steering_angle': steering_angle.__str__(),
    'throttle': throttle.__str__()
    }, skip_sid=True)



    
with open('model.json', 'r') as jfile:    
    model = model_from_json(jfile.read())


#model.compile("adam", "mse")
# load weights into new model
model.load_weights("model.h5")

# wrap Flask application with engineio's middleware
#app = socketio.Middleware(sio, app)

# deploy as an eventlet WSGI server
#eventlet.wsgi.server(eventlet.listen(('', 4567)), app)

TypeError: string indices must be integers